<a href="https://colab.research.google.com/github/sayanbanerjee32/keyword_search/blob/main/Keyword_search_fastai_LM.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
%%capture
# To import kaggle datasets
!pip install --upgrade pip 
!pip install kaggle==1.5.6

In [2]:
# import colab libraries
from google.colab import files
# import kaggle json to connect to kaggle user account to download datsets
files.upload()

Saving kaggle.json to kaggle.json


{'kaggle.json': b'{"username":"saansd2003","key":"49c2be11dfd314e10999ff432e1eeb65"}'}

In [3]:
# The Kaggle API client expects this file to be in ~/.kaggle,
# so lets move it there.
!mkdir ~/.kaggle
!cp kaggle.json ~/.kaggle/

In [4]:
# This permissions change avoids a warning on Kaggle tool startup.
!chmod 600 ~/.kaggle/kaggle.json

In [5]:
# download our dataset
!kaggle datasets download -d lakshmi25npathi/imdb-dataset-of-50k-movie-reviews -p 'datasets'

 19% 5.00M/25.7M [00:00<00:01, 19.1MB/s]
100% 25.7M/25.7M [00:00<00:00, 76.7MB/s]


In [6]:
#unzip
!unzip datasets/imdb-dataset-of-50k-movie-reviews.zip -d datasets/

Archive:  datasets/imdb-dataset-of-50k-movie-reviews.zip
  inflating: datasets/IMDB Dataset.csv  


In [7]:
%%capture
!pip install fastai -Uq
# !pip install rank_bm25 --quiet #install BM25
# !pip install --no-binary :all: nmslib #install nmslib
# !pip install clean-text[gpl]
# !pip install sentencepiece

In [8]:
import pandas as pd
from fastai.text.all import *

In [9]:
df = pd.read_csv('datasets/IMDB Dataset.csv')
df.shape

(50000, 2)

In [10]:
df.head(1)

,review,sentiment
0,"One of the other reviewers has mentioned that after watching just 1 Oz episode you'll be hooked. They are right, as this is exactly what happened with me.<br /><br />The first thing that struck me about Oz was its brutality and unflinching scenes of violence, which set in right from the word GO. Trust me, this is not a show for the faint hearted or timid. This show pulls no punches with regards to drugs, sex or violence. Its is hardcore, in the classic use of the word.<br /><br />It is called OZ as that is the nickname given to the Oswald Maximum Security State Penitentary. It focuses main...",positive


In [ ]:
dls_lm = DataBlock(blocks=TextBlock.from_df('review', is_lm=True),
                        splitter=RandomSplitter(0.1)).dataloaders(df, bs=128, seq_len=80)
dls_lm.show_batch(max_n=2)

In [ ]:
learn = language_model_learner(
    dls_lm, AWD_LSTM, drop_mult=0.3, 
    metrics=[accuracy, Perplexity()]).to_fp16()